In [1]:
pip install runpod pillow


  Using cached runpod-1.7.12-py3-none-any.whl.metadata (8.1 kB)
  Using cached aiohttp_retry-2.9.1-py3-none-any.whl.metadata (8.8 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached boto3-1.39.3-py3-none-any.whl.metadata (6.6 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached fastapi-0.115.14-py3-none-any.whl.metadata (27 kB)
  Using cached paramiko-3.5.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached prettytable-3.16.0-py3-none-any.whl.metadata (33 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached InquirerPy-0.3.4-py3-none-any.whl.metadata (8.1 kB)
  Using cached tomlkit-0.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached tqdm_loggable-0.2-py3-none-any.whl.metadata (9.3 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached pfzy-0.3.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached botocore-1.39.3-py3-none-any.whl.metadata (5.7 kB)
  Using cach

In [2]:
!pip install runpod pillow
!pip install waitress


In [3]:
# test_v7_fixed.py
import requests
import json  # FALTABA ESTE IMPORT
import base64
from PIL import Image
import io
import time

# Tu configuración
API_KEY = "rpa_B4ZHWTBS3ZFT8SMFCVQWTOWW1LWKG5K27DI5NI071k6jja" 
ENDPOINT_ID = "06nb0joagtej7p"

# Headers
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# Workflow completo
workflow = {
    "3": {
        "inputs": {
            "seed": 12345,
            "steps": 5,  # MÁS REDUCIDO para debug
            "cfg": 5,
            "sampler_name": "dpmpp_sde_gpu",
            "scheduler": "normal",
            "denoise": 1,
            "model": ["14", 0],
            "positive": ["6", 0],
            "negative": ["7", 0],
            "latent_image": ["5", 0]
        },
        "class_type": "KSampler"
    },
    "5": {
        "inputs": {
            "width": 512,  # REDUCIDO para test más rápido
            "height": 512,
            "batch_size": 1
        },
        "class_type": "EmptyLatentImage"
    },
    "6": {
        "inputs": {
            "text": "fortnite character",  # Prompt simple
            "clip": ["14", 1]
        },
        "class_type": "CLIPTextEncode"
    },
    "7": {
        "inputs": {
            "text": "photo",
            "clip": ["14", 1]
        },
        "class_type": "CLIPTextEncode"
    },
    "8": {
        "inputs": {
            "samples": ["3", 0],
            "vae": ["14", 2]
        },
        "class_type": "VAEDecode"
    },
    "9": {
        "inputs": {
            "filename_prefix": "API_v7",
            "images": ["8", 0]
        },
        "class_type": "SaveImage"
    },
    "14": {
        "inputs": {
            "model_path": "v1x0_fortnite_humanoid_sdxl1_vae_fix-000005"
        },
        "class_type": "DiffusersLoader"
    }
}

print("=== Test de API v7 ===")
print("Enviando request...")

# Enviar request
response = requests.post(
    f"https://api.runpod.ai/v2/{ENDPOINT_ID}/runsync",
    headers=headers,
    json={"input": {"workflow": workflow}},
    timeout=300
)

result = response.json()
print(f"\nRespuesta completa:")
print(json.dumps(result, indent=2))

# Procesar resultado
if result.get("output", {}).get("status") == "success":
    images = result["output"]["images"]
    print(f"\n✅ ÉXITO! {len(images)} imágenes")
    
    img_data = base64.b64decode(images[0])
    img = Image.open(io.BytesIO(img_data))
    img.save("test_v7.png")
    print("Imagen guardada!")
else:
    print(f"\n❌ Error: {result.get('output', {}).get('error', 'Ver arriba')}")

=== Test de API v7 ===
Enviando request...

Respuesta completa:
{
  "delayTime": 1082,
  "error": "HTTPConnectionPool(host='127.0.0.1', port=8188): Max retries exceeded with url: /prompt (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x73522a525780>: Failed to establish a new connection: [Errno 111] Connection refused'))",
  "executionTime": 30264,
  "id": "sync-0e997aea-8b2f-43fa-9778-ac5fa971b958-e1",
  "output": {
    "status": "failed"
  },
  "status": "FAILED",
  "workerId": "52mu7hzl7cd3ux"
}

❌ Error: Ver arriba
